Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [7]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-12-18 18:23:27,010: INFO: running C:\Users\putma\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py --f=c:\Users\putma\AppData\Roaming\jupyter\runtime\kernel-v2-43682scn1LylnGvL.json
2023-12-18 18:24:00,269: INFO: Saved 10000 articles
2023-12-18 18:24:22,836: INFO: Saved 20000 articles
2023-12-18 18:24:43,023: INFO: Saved 30000 articles
2023-12-18 18:24:59,349: INFO: Saved 40000 articles
2023-12-18 18:25:17,407: INFO: Saved 50000 articles
2023-12-18 18:25:33,768: INFO: Saved 60000 articles
2023-12-18 18:25:49,999: INFO: Saved 70000 articles
2023-12-18 18:26:07,287: INFO: Saved 80000 articles
2023-12-18 18:26:32,223: INFO: Saved 90000 articles
2023-12-18 18:26:50,229: INFO: Saved 100000 articles
2023-12-18 18:27:07,037: INFO: Saved 110000 articles
2023-12-18 18:27:22,136: INFO: Saved 120000 articles
2023-12-18 18:27:44,081: INFO: Saved 130000 articles
2023-12-18 18:28:01,302: INFO: Saved 140000 articles
2023-12-18 18:28:12,763: INFO: Saved 150000 articles
2023-12-

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [8]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-12-18 19:54:11,071: INFO: running C:\Users\putma\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py --f=c:\Users\putma\AppData\Roaming\jupyter\runtime\kernel-v2-43682scn1LylnGvL.json
2023-12-18 19:54:11,074: INFO: collecting all words and their counts
2023-12-18 19:54:11,089: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-12-18 19:54:15,201: INFO: PROGRESS: at sentence #10000, processed 10772827 words, keeping 354272 word types
2023-12-18 19:54:17,725: INFO: PROGRESS: at sentence #20000, processed 17387158 words, keeping 490949 word types
2023-12-18 19:54:19,535: INFO: PROGRESS: at sentence #30000, processed 22203855 words, keeping 591750 word types
2023-12-18 19:54:20,924: INFO: PROGRESS: at sentence #40000, processed 26618476 words, keeping 676712 word types
2023-12-18 19:54:22,438: INFO: PROGRESS: at sentence #50000, processed 31364477 words, keeping 775894 word types
2023-12-18 19:54:23,679: INFO: PROGRESS: at sentence #60000, pr

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [9]:
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Kemang', 0.6136735677719116), ('Surabaya', 0.6059540510177612), ('Warteg', 0.6048282384872437), ('Cilandak', 0.5947189331054688), ('Warkop', 0.5758446455001831), ('Ciputat', 0.5735667943954468), ('Kemayoran', 0.564713716506958), ('WARTEG', 0.5626453161239624), ('Epicentrum', 0.5579254031181335), ('Condet', 0.5516181588172913)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7075418829917908), ('Bogor', 0.6942011117935181), ('Tasikmalaya', 0.6320353150367737), ('Sukabumi', 0.6307504773139954), ('Garut', 0.6147168874740601), ('Baleendah', 0.6120404005050659), ('Cianjur', 0.6031326055526733), ('Jatinangor', 0.600104808807373), ('Purwakarta', 0.5956789255142212), ('Majalaya', 0.589984655380249)]
Kedekatan Yogyakarta-Surakarta: 0.795225203037262
Kedekatan Yogyakarta-Semarang: 0.6368850469589233
makanan-rendang, minuman-?: [('martabak', 0.8695498038248111), ('Fanta', 0.8580974605174111), ('wedang', 0.8529131784609358), ('puding', 0.851005173717337), ('Minuman', 0